In [16]:
import json
import torch
from torchvision.datasets import ImageFolder,DatasetFolder
import model
from torchvision import transforms
from sentence_transformers import SentenceTransformer
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.models as models

In [24]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
cap = dset.CocoCaptions(root = '/data/coco/train2017',
                        annFile = '/data/coco/annotations/captions_train2017.json',
                        transform=transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.ToTensor(),
            normalize,
        ]))

loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


In [25]:
img, txt = cap[0]

In [26]:
txt

['Closeup of bins of food that include broccoli and bread.',
 'A meal is presented in brightly colored plastic trays.',
 'there are containers filled with different kinds of foods',
 'Colorful dishes holding meat, vegetables, fruit, and bread.',
 'A bunch of trays that have different food.']

In [27]:
path = "/home/super/code/DSCMR/pretrain_model/moco_v2_800ep_pretrain.pth.tar"
model = models.__dict__['resnet50'](num_classes=128)
checkpoint = torch.load(path, map_location="cpu")

state_dict = checkpoint['state_dict']
for k in list(state_dict.keys()):
    # retain only encoder_q up to before the embedding layer
    if k.startswith('module.encoder_q') and not k.startswith('module.encoder_q.fc'):
        # remove prefix
        state_dict[k[len("module.encoder_q."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]

model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=[])

In [29]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
out = model(torch.unsqueeze(img, 0))
out.shape

torch.Size([1, 128])

In [ ]:
class Datasetstr(object):
    def __init__(self, path):
        f = open(path)
        load_dict = json.load(f)
        self.txt = ['0'] * len(load_dict['images']) * 5
        self.label_txt = np.zeros(len(load_dict['images']) * 5)
        for i in range(len(load_dict['images'])):
            for j in range(5):
                txt[i * 5 + j] = load_dict['images'][i]['sentences'][j]['raw']
                label_txt[i * 5 + j] = int(load_dict['images'][i]['imgid'])

    def __getitem__(self, index):
        return self.txt[index], self.label_txt[index]

    def __len__(self):
        return len(self.txt)

In [ ]:
train_set = Datasetstr('/data/caption_datasets/dataset_flickr30k.json')
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=False)

In [6]:
device = torch.device("cuda")
#model_img = model.VGGNet().to(device)
model_txt = SentenceTransformer('bert-large-nli-stsb-mean-tokens').to(device)

f = open('/data/caption_datasets/dataset_flickr30k.json')
load_dict = json.load(f)

In [19]:
#print(load_dict['images'][105].keys())
print((load_dict['images'][105]['filename']))

1030985833.jpg


31014

In [4]:
import numpy as np
embedding_txt = np.zeros((len(load_dict['images']), 1024))
label_txt = np.zeros(len(load_dict['images']))

In [7]:
for i in range(len(load_dict['images'])):
    print(i)
    for j in range(1):
        sentence_embeddings = model_txt.encode(load_dict['images'][i]['sentences'][j]['raw'])
        embedding_txt[i] = sentence_embeddings
        label_txt[i] = int(load_dict['images'][i]['imgid'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066


3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712


5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355


6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995


...

25147
25148
25149
25150
25151
25152
25153
25154
25155
25156
25157
25158
25159
25160
25161
25162
25163
25164
25165
25166
25167
25168
25169
25170
25171
25172
25173
25174
25175
25176
25177
25178
25179
25180
25181
25182
25183
25184
25185
25186
25187
25188
25189
25190
25191
25192
25193
25194
25195
25196
25197
25198
25199
25200
25201
25202
25203
25204
25205
25206
25207
25208
25209
25210
25211
25212
25213
25214
25215
25216
25217
25218
25219
25220
25221
25222
25223
25224
25225
25226
25227
25228
25229
25230
25231
25232
25233
25234
25235
25236
25237
25238
25239
25240
25241
25242
25243
25244
25245
25246
25247
25248
25249
25250
25251
25252
25253
25254
25255
25256
25257
25258
25259
25260
25261
25262
25263
25264
25265
25266
25267
25268
25269
25270
25271
25272
25273
25274
25275
25276
25277
25278
25279
25280
25281
25282
25283
25284
25285
25286
25287
25288
25289
25290
25291
25292
25293
25294
25295
25296
25297
25298
25299
25300
25301
25302
25303
25304
25305
25306
25307
25308
25309
25310
25311
25312
2531

26517
26518
26519
26520
26521
26522
26523
26524
26525
26526
26527
26528
26529
26530
26531
26532
26533
26534
26535
26536
26537
26538
26539
26540
26541
26542
26543
26544
26545
26546
26547
26548
26549
26550
26551
26552
26553
26554
26555
26556
26557
26558
26559
26560
26561
26562
26563
26564
26565
26566
26567
26568
26569
26570
26571
26572
26573
26574
26575
26576
26577
26578
26579
26580
26581
26582
26583
26584
26585
26586
26587
26588
26589
26590
26591
26592
26593
26594
26595
26596
26597
26598
26599
26600
26601
26602
26603
26604
26605
26606
26607
26608
26609
26610
26611
26612
26613
26614
26615
26616
26617
26618
26619
26620
26621
26622
26623
26624
26625
26626
26627
26628
26629
26630
26631
26632
26633
26634
26635
26636
26637
26638
26639
26640
26641
26642
26643
26644
26645
26646
26647
26648
26649
26650
26651
26652
26653
26654
26655
26656
26657
26658
26659
26660
26661
26662
26663
26664
26665
26666
26667
26668
26669
26670
26671
26672
26673
26674
26675
26676
26677
26678
26679
26680
26681
26682
2668

27890
27891
27892
27893
27894
27895
27896
27897
27898
27899
27900
27901
27902
27903
27904
27905
27906
27907
27908
27909
27910
27911
27912
27913
27914
27915
27916
27917
27918
27919
27920
27921
27922
27923
27924
27925
27926
27927
27928
27929
27930
27931
27932
27933
27934
27935
27936
27937
27938
27939
27940
27941
27942
27943
27944
27945
27946
27947
27948
27949
27950
27951
27952
27953
27954
27955
27956
27957
27958
27959
27960
27961
27962
27963
27964
27965
27966
27967
27968
27969
27970
27971
27972
27973
27974
27975
27976
27977
27978
27979
27980
27981
27982
27983
27984
27985
27986
27987
27988
27989
27990
27991
27992
27993
27994
27995
27996
27997
27998
27999
28000
28001
28002
28003
28004
28005
28006
28007
28008
28009
28010
28011
28012
28013
28014
28015
28016
28017
28018
28019
28020
28021
28022
28023
28024
28025
28026
28027
28028
28029
28030
28031
28032
28033
28034
28035
28036
28037
28038
28039
28040
28041
28042
28043
28044
28045
28046
28047
28048
28049
28050
28051
28052
28053
28054
28055
2805

29256
29257
29258
29259
29260
29261
29262
29263
29264
29265
29266
29267
29268
29269
29270
29271
29272
29273
29274
29275
29276
29277
29278
29279
29280
29281
29282
29283
29284
29285
29286
29287
29288
29289
29290
29291
29292
29293
29294
29295
29296
29297
29298
29299
29300
29301
29302
29303
29304
29305
29306
29307
29308
29309
29310
29311
29312
29313
29314
29315
29316
29317
29318
29319
29320
29321
29322
29323
29324
29325
29326
29327
29328
29329
29330
29331
29332
29333
29334
29335
29336
29337
29338
29339
29340
29341
29342
29343
29344
29345
29346
29347
29348
29349
29350
29351
29352
29353
29354
29355
29356
29357
29358
29359
29360
29361
29362
29363
29364
29365
29366
29367
29368
29369
29370
29371
29372
29373
29374
29375
29376
29377
29378
29379
29380
29381
29382
29383
29384
29385
29386
29387
29388
29389
29390
29391
29392
29393
29394
29395
29396
29397
29398
29399
29400
29401
29402
29403
29404
29405
29406
29407
29408
29409
29410
29411
29412
29413
29414
29415
29416
29417
29418
29419
29420
29421
2942

30623
30624
30625
30626
30627
30628
30629
30630
30631
30632
30633
30634
30635
30636
30637
30638
30639
30640
30641
30642
30643
30644
30645
30646
30647
30648
30649
30650
30651
30652
30653
30654
30655
30656
30657
30658
30659
30660
30661
30662
30663
30664
30665
30666
30667
30668
30669
30670
30671
30672
30673
30674
30675
30676
30677
30678
30679
30680
30681
30682
30683
30684
30685
30686
30687
30688
30689
30690
30691
30692
30693
30694
30695
30696
30697
30698
30699
30700
30701
30702
30703
30704
30705
30706
30707
30708
30709
30710
30711
30712
30713
30714
30715
30716
30717
30718
30719
30720
30721
30722
30723
30724
30725
30726
30727
30728
30729
30730
30731
30732
30733
30734
30735
30736
30737
30738
30739
30740
30741
30742
30743
30744
30745
30746
30747
30748
30749
30750
30751
30752
30753
30754
30755
30756
30757
30758
30759
30760
30761
30762
30763
30764
30765
30766
30767
30768
30769
30770
30771
30772
30773
30774
30775
30776
30777
30778
30779
30780
30781
30782
30783
30784
30785
30786
30787
30788
3078

In [48]:
for i in range(0, len(load_dict['images'])):
    inp = []
    print(i)
    for j in range(5):
        inp += [load_dict['images'][i]['sentences'][j]['raw']]
    sentence_embeddings = model_txt.encode(inp)
    embedding_txt[i * 5 : i * 5 + 5] = sentence_embeddings
    label_txt[i * 5 : i * 5 + 5] = int(load_dict['images'][i]['imgid'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704


5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349


6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994


...

25156
25157
25158
25159
25160
25161
25162
25163
25164
25165
25166
25167
25168
25169
25170
25171
25172
25173
25174
25175
25176
25177
25178
25179
25180
25181
25182
25183
25184
25185
25186
25187
25188
25189
25190
25191
25192
25193
25194
25195
25196
25197
25198
25199
25200
25201
25202
25203
25204
25205
25206
25207
25208
25209
25210
25211
25212
25213
25214
25215
25216
25217
25218
25219
25220
25221
25222
25223
25224
25225
25226
25227
25228
25229
25230
25231
25232
25233
25234
25235
25236
25237
25238
25239
25240
25241
25242
25243
25244
25245
25246
25247
25248
25249
25250
25251
25252
25253
25254
25255
25256
25257
25258
25259
25260
25261
25262
25263
25264
25265
25266
25267
25268
25269
25270
25271
25272
25273
25274
25275
25276
25277
25278
25279
25280
25281
25282
25283
25284
25285
25286
25287
25288
25289
25290
25291
25292
25293
25294
25295
25296
25297
25298
25299
25300
25301
25302
25303
25304
25305
25306
25307
25308
25309
25310
25311
25312
25313
25314
25315
25316
25317
25318
25319
25320
25321
2532

26528
26529
26530
26531
26532
26533
26534
26535
26536
26537
26538
26539
26540
26541
26542
26543
26544
26545
26546
26547
26548
26549
26550
26551
26552
26553
26554
26555
26556
26557
26558
26559
26560
26561
26562
26563
26564
26565
26566
26567
26568
26569
26570
26571
26572
26573
26574
26575
26576
26577
26578
26579
26580
26581
26582
26583
26584
26585
26586
26587
26588
26589
26590
26591
26592
26593
26594
26595
26596
26597
26598
26599
26600
26601
26602
26603
26604
26605
26606
26607
26608
26609
26610
26611
26612
26613
26614
26615
26616
26617
26618
26619
26620
26621
26622
26623
26624
26625
26626
26627
26628
26629
26630
26631
26632
26633
26634
26635
26636
26637
26638
26639
26640
26641
26642
26643
26644
26645
26646
26647
26648
26649
26650
26651
26652
26653
26654
26655
26656
26657
26658
26659
26660
26661
26662
26663
26664
26665
26666
26667
26668
26669
26670
26671
26672
26673
26674
26675
26676
26677
26678
26679
26680
26681
26682
26683
26684
26685
26686
26687
26688
26689
26690
26691
26692
26693
2669

27898
27899
27900
27901
27902
27903
27904
27905
27906
27907
27908
27909
27910
27911
27912
27913
27914
27915
27916
27917
27918
27919
27920
27921
27922
27923
27924
27925
27926
27927
27928
27929
27930
27931
27932
27933
27934
27935
27936
27937
27938
27939
27940
27941
27942
27943
27944
27945
27946
27947
27948
27949
27950
27951
27952
27953
27954
27955
27956
27957
27958
27959
27960
27961
27962
27963
27964
27965
27966
27967
27968
27969
27970
27971
27972
27973
27974
27975
27976
27977
27978
27979
27980
27981
27982
27983
27984
27985
27986
27987
27988
27989
27990
27991
27992
27993
27994
27995
27996
27997
27998
27999
28000
28001
28002
28003
28004
28005
28006
28007
28008
28009
28010
28011
28012
28013
28014
28015
28016
28017
28018
28019
28020
28021
28022
28023
28024
28025
28026
28027
28028
28029
28030
28031
28032
28033
28034
28035
28036
28037
28038
28039
28040
28041
28042
28043
28044
28045
28046
28047
28048
28049
28050
28051
28052
28053
28054
28055
28056
28057
28058
28059
28060
28061
28062
28063
2806

29270
29271
29272
29273
29274
29275
29276
29277
29278
29279
29280
29281
29282
29283
29284
29285
29286
29287
29288
29289
29290
29291
29292
29293
29294
29295
29296
29297
29298
29299
29300
29301
29302
29303
29304
29305
29306
29307
29308
29309
29310
29311
29312
29313
29314
29315
29316
29317
29318
29319
29320
29321
29322
29323
29324
29325
29326
29327
29328
29329
29330
29331
29332
29333
29334
29335
29336
29337
29338
29339
29340
29341
29342
29343
29344
29345
29346
29347
29348
29349
29350
29351
29352
29353
29354
29355
29356
29357
29358
29359
29360
29361
29362
29363
29364
29365
29366
29367
29368
29369
29370
29371
29372
29373
29374
29375
29376
29377
29378
29379
29380
29381
29382
29383
29384
29385
29386
29387
29388
29389
29390
29391
29392
29393
29394
29395
29396
29397
29398
29399
29400
29401
29402
29403
29404
29405
29406
29407
29408
29409
29410
29411
29412
29413
29414
29415
29416
29417
29418
29419
29420
29421
29422
29423
29424
29425
29426
29427
29428
29429
29430
29431
29432
29433
29434
29435
2943

30642
30643
30644
30645
30646
30647
30648
30649
30650
30651
30652
30653
30654
30655
30656
30657
30658
30659
30660
30661
30662
30663
30664
30665
30666
30667
30668
30669
30670
30671
30672
30673
30674
30675
30676
30677
30678
30679
30680
30681
30682
30683
30684
30685
30686
30687
30688
30689
30690
30691
30692
30693
30694
30695
30696
30697
30698
30699
30700
30701
30702
30703
30704
30705
30706
30707
30708
30709
30710
30711
30712
30713
30714
30715
30716
30717
30718
30719
30720
30721
30722
30723
30724
30725
30726
30727
30728
30729
30730
30731
30732
30733
30734
30735
30736
30737
30738
30739
30740
30741
30742
30743
30744
30745
30746
30747
30748
30749
30750
30751
30752
30753
30754
30755
30756
30757
30758
30759
30760
30761
30762
30763
30764
30765
30766
30767
30768
30769
30770
30771
30772
30773
30774
30775
30776
30777
30778
30779
30780
30781
30782
30783
30784
30785
30786
30787
30788
30789
30790
30791
30792
30793
30794
30795
30796
30797
30798
30799
30800
30801
30802
30803
30804
30805
30806
30807
3080

In [8]:
np.save("flick30k_embedding_txt.npy", embedding_txt)
np.save( "flick30k_label_txt.npy", label_txt)

In [3]:
import cv2
import numpy as np
import torchvision
train_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

In [13]:
class Datasetimg(object):
    def __init__(self, path):
        f = open(path)
        load_dict = json.load(f)
        #self.txt = ['0'] * len(load_dict['images']) * 5
        self.name = []
        self.label_img = np.zeros(len(load_dict['images']))
        print(len(load_dict['images']))
        self.dim = (224, 224)
        for i in range(len(load_dict['images'])):
            self.name.append(load_dict['images'][i]['filename'])
            self.label_img[i] = int(load_dict['images'][i]['imgid'])

    def __getitem__(self, index):
        img = cv2.imread('/data/flickr30k/flickr30k-images/'+ self.name[index])
        print('/data/flickr30k/flickr30k-images/'+ self.name[index])
        #img1 = cv2.rotate(img, cv2.ROTATE_180)
        #img2 = cv2.GaussianBlur(img,(3,3),0)
        #img3 = cv2.GaussianBlur(img,(1,1),0)
        #img4 = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.resize(img, self.dim, interpolation = cv2.INTER_AREA)
        #img1 = cv2.resize(img1, self.dim, interpolation = cv2.INTER_AREA)
        #img2 = cv2.resize(img2, self.dim, interpolation = cv2.INTER_AREA)
        #img3 = cv2.resize(img3, self.dim, interpolation = cv2.INTER_AREA)
        #img4 = cv2.resize(img4, self.dim, interpolation = cv2.INTER_AREA)
        img=cv2.normalize(img, None, norm_type=cv2.NORM_MINMAX)
        #img1=cv2.normalize(img1, None, norm_type=cv2.NORM_MINMAX)
        #img2=cv2.normalize(img2, None, norm_type=cv2.NORM_MINMAX)
        #img3=cv2.normalize(img3, None, norm_type=cv2.NORM_MINMAX)
        #img4=cv2.normalize(img4, None, norm_type=cv2.NORM_MINMAX)
        #out = np.concatenate((img, img1), axis=0)
        #out = np.concatenate((out, img2), axis=0)
        #out = np.concatenate((out, img3), axis=0)
        #out = np.concatenate((out, img4), axis=0)
        #out = np.array([img,img1,img2,img3,img4][:],dtype=np.float32)#torch.FloatTensor([img,img1,img2,img3,img4])
        return torch.from_numpy(img).float(), self.label_img[index]

    def __len__(self):
        return len(self.name)

In [14]:
device = torch.device("cuda")
model_img = model.VGGNet().to(device)
train_set = Datasetimg('/data/caption_datasets/dataset_flickr30k.json')
train_loader = torch.utils.data.DataLoader(train_set, batch_size=40, shuffle=False)
model_img.eval()

31014


VGGNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 

In [6]:
embedding_img = np.zeros((31014, 2048))

In [7]:
embedding_img.shape

(31014, 2048)

In [15]:
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    print(data.shape)
    #data = torch.squeeze(data)
    #data = torch.reshape(data, (data.shape[0], 224, 224, 3))
    #print(data.shape,target.shape)
    data = data.permute(0, 3, 1, 2)
    output = model_img(data)
    embedding_img[batch_idx*40 : batch_idx*40 + output.shape[0]] = output.cpu().detach().numpy()
    print(batch_idx*40, batch_idx*40 + output.shape[0], output.shape)
    #print(output)
    

/data/flickr30k/flickr30k-images/1000092795.jpg
/data/flickr30k/flickr30k-images/10002456.jpg
/data/flickr30k/flickr30k-images/1000268201.jpg
/data/flickr30k/flickr30k-images/1000344755.jpg
/data/flickr30k/flickr30k-images/1000366164.jpg
/data/flickr30k/flickr30k-images/1000523639.jpg
/data/flickr30k/flickr30k-images/1000919630.jpg
/data/flickr30k/flickr30k-images/10010052.jpg
/data/flickr30k/flickr30k-images/1001465944.jpg
/data/flickr30k/flickr30k-images/1001545525.jpg
/data/flickr30k/flickr30k-images/1001573224.jpg
/data/flickr30k/flickr30k-images/1001633352.jpg
/data/flickr30k/flickr30k-images/1001773457.jpg
/data/flickr30k/flickr30k-images/1001896054.jpg
/data/flickr30k/flickr30k-images/100197432.jpg
/data/flickr30k/flickr30k-images/100207720.jpg
/data/flickr30k/flickr30k-images/1002674143.jpg
/data/flickr30k/flickr30k-images/1003163366.jpg
/data/flickr30k/flickr30k-images/1003420127.jpg
/data/flickr30k/flickr30k-images/1003428081.jpg
/data/flickr30k/flickr30k-images/100444898.jpg

/data/flickr30k/flickr30k-images/1066831905.jpg
/data/flickr30k/flickr30k-images/106691539.jpg
/data/flickr30k/flickr30k-images/1067180831.jpg
/data/flickr30k/flickr30k-images/1067790824.jpg
/data/flickr30k/flickr30k-images/1068427675.jpg
/data/flickr30k/flickr30k-images/106959209.jpg
/data/flickr30k/flickr30k-images/107022557.jpg
/data/flickr30k/flickr30k-images/1071201387.jpg
torch.Size([40, 224, 224, 3])
160 200 torch.Size([40, 2048])
/data/flickr30k/flickr30k-images/1072439304.jpg
/data/flickr30k/flickr30k-images/1073145694.jpg
/data/flickr30k/flickr30k-images/107318069.jpg
/data/flickr30k/flickr30k-images/1073444492.jpg
/data/flickr30k/flickr30k-images/107368840.jpg
/data/flickr30k/flickr30k-images/10751489.jpg
/data/flickr30k/flickr30k-images/107532584.jpg
/data/flickr30k/flickr30k-images/1075716537.jpg
/data/flickr30k/flickr30k-images/107582366.jpg
/data/flickr30k/flickr30k-images/1075867198.jpg
/data/flickr30k/flickr30k-images/1075881101.jpg
/data/flickr30k/flickr30k-images/107

/data/flickr30k/flickr30k-images/1122579380.jpg
/data/flickr30k/flickr30k-images/112264837.jpg
/data/flickr30k/flickr30k-images/1122944218.jpg
/data/flickr30k/flickr30k-images/1123486015.jpg
/data/flickr30k/flickr30k-images/112411540.jpg
/data/flickr30k/flickr30k-images/1124448967.jpg
/data/flickr30k/flickr30k-images/11245644.jpg
/data/flickr30k/flickr30k-images/112459543.jpg
/data/flickr30k/flickr30k-images/112550433.jpg
/data/flickr30k/flickr30k-images/112604461.jpg
/data/flickr30k/flickr30k-images/1126137905.jpg
/data/flickr30k/flickr30k-images/1126403094.jpg
/data/flickr30k/flickr30k-images/1126450961.jpg
/data/flickr30k/flickr30k-images/112719695.jpg
/data/flickr30k/flickr30k-images/1128230658.jpg
/data/flickr30k/flickr30k-images/1128874064.jpg
/data/flickr30k/flickr30k-images/112945950.jpg
/data/flickr30k/flickr30k-images/1129704496.jpg
/data/flickr30k/flickr30k-images/1130017585.jpg
/data/flickr30k/flickr30k-images/1130295306.jpg
/data/flickr30k/flickr30k-images/113032513.jpg
/d

/data/flickr30k/flickr30k-images/1186603607.jpg
/data/flickr30k/flickr30k-images/118667747.jpg
/data/flickr30k/flickr30k-images/118668756.jpg
/data/flickr30k/flickr30k-images/1187077747.jpg
/data/flickr30k/flickr30k-images/118707970.jpg
/data/flickr30k/flickr30k-images/118708295.jpg
/data/flickr30k/flickr30k-images/118717792.jpg
/data/flickr30k/flickr30k-images/1187252153.jpg
/data/flickr30k/flickr30k-images/1187435567.jpg
/data/flickr30k/flickr30k-images/1187593464.jpg
torch.Size([40, 224, 224, 3])
520 560 torch.Size([40, 2048])
/data/flickr30k/flickr30k-images/118771635.jpg
/data/flickr30k/flickr30k-images/1187993157.jpg
/data/flickr30k/flickr30k-images/118802644.jpg
/data/flickr30k/flickr30k-images/1188199432.jpg
/data/flickr30k/flickr30k-images/118854260.jpg
/data/flickr30k/flickr30k-images/118865844.jpg
/data/flickr30k/flickr30k-images/1188670028.jpg
/data/flickr30k/flickr30k-images/118897631.jpg
/data/flickr30k/flickr30k-images/118939364.jpg
/data/flickr30k/flickr30k-images/11893

680 720 torch.Size([40, 2048])
/data/flickr30k/flickr30k-images/124534466.jpg
/data/flickr30k/flickr30k-images/1246237064.jpg
/data/flickr30k/flickr30k-images/1246239396.jpg
/data/flickr30k/flickr30k-images/1246863003.jpg
/data/flickr30k/flickr30k-images/1247181182.jpg
/data/flickr30k/flickr30k-images/1247484195.jpg
/data/flickr30k/flickr30k-images/1248357227.jpg
/data/flickr30k/flickr30k-images/1248734482.jpg
/data/flickr30k/flickr30k-images/1248747267.jpg
/data/flickr30k/flickr30k-images/124880336.jpg
/data/flickr30k/flickr30k-images/124881487.jpg
/data/flickr30k/flickr30k-images/1248940539.jpg
/data/flickr30k/flickr30k-images/1248953128.jpg
/data/flickr30k/flickr30k-images/124972799.jpg
/data/flickr30k/flickr30k-images/1250181412.jpg
/data/flickr30k/flickr30k-images/1250745375.jpg
/data/flickr30k/flickr30k-images/1250845715.jpg
/data/flickr30k/flickr30k-images/1250854171.jpg
/data/flickr30k/flickr30k-images/125093729.jpg
/data/flickr30k/flickr30k-images/1251558317.jpg
/data/flickr30

...

torch.Size([40, 224, 224, 3])
30240 30280 torch.Size([40, 2048])
/data/flickr30k/flickr30k-images/8051568083.jpg
/data/flickr30k/flickr30k-images/8052328504.jpg
/data/flickr30k/flickr30k-images/8052380671.jpg
/data/flickr30k/flickr30k-images/8052387530.jpg
/data/flickr30k/flickr30k-images/8052391339.jpg
/data/flickr30k/flickr30k-images/8052397902.jpg
/data/flickr30k/flickr30k-images/8052575239.jpg
/data/flickr30k/flickr30k-images/8053033868.jpg
/data/flickr30k/flickr30k-images/805682444.jpg
/data/flickr30k/flickr30k-images/8057415401.jpg
/data/flickr30k/flickr30k-images/8058635660.jpg
/data/flickr30k/flickr30k-images/8060043940.jpg
/data/flickr30k/flickr30k-images/8060276063.jpg
/data/flickr30k/flickr30k-images/8063007.jpg
/data/flickr30k/flickr30k-images/8063022480.jpg
/data/flickr30k/flickr30k-images/8063802042.jpg
/data/flickr30k/flickr30k-images/8063954905.jpg
/data/flickr30k/flickr30k-images/8064150004.jpg
/data/flickr30k/flickr30k-images/8064982845.jpg
/data/flickr30k/flickr30k-i

/data/flickr30k/flickr30k-images/8179518136.jpg
/data/flickr30k/flickr30k-images/8180138587.jpg
/data/flickr30k/flickr30k-images/8180474484.jpg
/data/flickr30k/flickr30k-images/8182074427.jpg
/data/flickr30k/flickr30k-images/818279377.jpg
/data/flickr30k/flickr30k-images/8183107966.jpg
/data/flickr30k/flickr30k-images/818340833.jpg
/data/flickr30k/flickr30k-images/8183834568.jpg
/data/flickr30k/flickr30k-images/8184925816.jpg
/data/flickr30k/flickr30k-images/8186248597.jpg
/data/flickr30k/flickr30k-images/8187910453.jpg
/data/flickr30k/flickr30k-images/8188882833.jpg
/data/flickr30k/flickr30k-images/8189395281.jpg
/data/flickr30k/flickr30k-images/8190802361.jpg
/data/flickr30k/flickr30k-images/8192398089.jpg
/data/flickr30k/flickr30k-images/8192488887.jpg
/data/flickr30k/flickr30k-images/8194331212.jpg
/data/flickr30k/flickr30k-images/819526898.jpg
/data/flickr30k/flickr30k-images/819604884.jpg
/data/flickr30k/flickr30k-images/8196866291.jpg
/data/flickr30k/flickr30k-images/8198089950.

30600 30640 torch.Size([40, 2048])
/data/flickr30k/flickr30k-images/86112808.jpg
/data/flickr30k/flickr30k-images/86112875.jpg
/data/flickr30k/flickr30k-images/86112925.jpg
/data/flickr30k/flickr30k-images/86112958.jpg
/data/flickr30k/flickr30k-images/86120374.jpg
/data/flickr30k/flickr30k-images/86120427.jpg
/data/flickr30k/flickr30k-images/86120479.jpg
/data/flickr30k/flickr30k-images/86120569.jpg
/data/flickr30k/flickr30k-images/86120657.jpg
/data/flickr30k/flickr30k-images/86120682.jpg
/data/flickr30k/flickr30k-images/86120741.jpg
/data/flickr30k/flickr30k-images/86120813.jpg
/data/flickr30k/flickr30k-images/86120845.jpg
/data/flickr30k/flickr30k-images/86120876.jpg
/data/flickr30k/flickr30k-images/86120978.jpg
/data/flickr30k/flickr30k-images/86124605.jpg
/data/flickr30k/flickr30k-images/86127083.jpg
/data/flickr30k/flickr30k-images/86131617.jpg
/data/flickr30k/flickr30k-images/86131989.jpg
/data/flickr30k/flickr30k-images/86132021.jpg
/data/flickr30k/flickr30k-images/861661418.jp

/data/flickr30k/flickr30k-images/91972799.jpg
/data/flickr30k/flickr30k-images/921190481.jpg
/data/flickr30k/flickr30k-images/92246314.jpg
/data/flickr30k/flickr30k-images/92289076.jpg
/data/flickr30k/flickr30k-images/923550133.jpg
/data/flickr30k/flickr30k-images/92413203.jpg
/data/flickr30k/flickr30k-images/92497263.jpg
/data/flickr30k/flickr30k-images/925491651.jpg
/data/flickr30k/flickr30k-images/92679312.jpg
/data/flickr30k/flickr30k-images/9268793.jpg
/data/flickr30k/flickr30k-images/92732320.jpg
/data/flickr30k/flickr30k-images/927420680.jpg
/data/flickr30k/flickr30k-images/929679367.jpg
/data/flickr30k/flickr30k-images/930126921.jpg
/data/flickr30k/flickr30k-images/930748509.jpg
/data/flickr30k/flickr30k-images/931028778.jpg
/data/flickr30k/flickr30k-images/931156700.jpg
/data/flickr30k/flickr30k-images/9324151.jpg
/data/flickr30k/flickr30k-images/93270842.jpg
/data/flickr30k/flickr30k-images/93307139.jpg
/data/flickr30k/flickr30k-images/93311089.jpg
/data/flickr30k/flickr30k-i

torch.Size([40, 224, 224, 3])
30960 31000 torch.Size([40, 2048])
/data/flickr30k/flickr30k-images/99171998.jpg
/data/flickr30k/flickr30k-images/991971859.jpg
/data/flickr30k/flickr30k-images/99458430.jpg
/data/flickr30k/flickr30k-images/9950858.jpg
/data/flickr30k/flickr30k-images/9950913.jpg
/data/flickr30k/flickr30k-images/996089206.jpg
/data/flickr30k/flickr30k-images/996712323.jpg
/data/flickr30k/flickr30k-images/99679241.jpg
/data/flickr30k/flickr30k-images/996962407.jpg
/data/flickr30k/flickr30k-images/997338199.jpg
/data/flickr30k/flickr30k-images/997722733.jpg
/data/flickr30k/flickr30k-images/997876722.jpg
/data/flickr30k/flickr30k-images/99804383.jpg
/data/flickr30k/flickr30k-images/998845445.jpg
torch.Size([14, 224, 224, 3])
31000 31014 torch.Size([14, 2048])


In [16]:
embedding_img.shape

(31014, 2048)

In [17]:
embedding_img

array([[0.04527703, 0.3591122 , 0.04109187, ..., 0.46004984, 0.30739206,
        0.19399664],
       [0.96036893, 1.10346425, 0.06479366, ..., 0.24243185, 0.02460748,
        0.52881187],
       [0.34734333, 0.13335206, 0.33090132, ..., 0.28078622, 0.02705601,
        0.33912635],
       ...,
       [0.37171581, 0.20221506, 0.18394065, ..., 0.18317708, 0.06029556,
        0.54563725],
       [0.27012557, 0.18703365, 0.21044771, ..., 0.34803483, 0.02172274,
        0.21217425],
       [0.12565225, 0.07789625, 0.46284226, ..., 0.41410333, 0.1142013 ,
        0.45860079]])

In [18]:
np.save("flick30k_embedding_img.npy", embedding_img)

In [5]:
embedding_img = np.load("flick30k_embedding_img.npy")
embedding_txt = np.load("flick30k_embedding_txt.npy")
embedding_lab = np.load("flick30k_label_txt.npy")

In [6]:
embedding_txt.shape

(155070, 1024)

In [7]:
index = np.arange(155070)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test= train_test_split(index,test_size=0.01, random_state=42)

In [10]:
train_img, test_img = embedding_img[X_train], embedding_img[X_test]
train_txt, test_txt = embedding_txt[X_train], embedding_txt[X_test]
train_lab, test_lab = embedding_lab[X_train], embedding_lab[X_test]

In [11]:
import scipy.io as sio

In [12]:
path = "/home/super/code/DSCMR/data/flicker30k/"
sio.savemat(path + "train_img.mat", {'train_img': train_img})
sio.savemat(path + "test_img.mat", {'test_img': test_img})
sio.savemat(path + "train_txt.mat", {'train_txt': train_txt})
sio.savemat(path + "test_txt.mat", {'test_txt': test_txt})
sio.savemat(path + "train_img_lab.mat", {'train_img_lab': train_lab})
sio.savemat(path + "test_img_lab.mat", {'test_img_lab': test_lab})